## Introduction
해당 쥬피터파일에서는 기존의 이론에서 공부했던 방법론들에 대해 살펴보는 시간을 가져보겠습니다. 
- Apriori Algorithm 
- FP Growth 
- TF-IDF
- Word2Vec
- KNN Neareast Algorithm 
- SGD
- ALS 

In [1]:
import mlxtend
import sklearn
import pandas as pd
import numpy as np
import gensim
import implicit
import surprise

## 1. Apriori 알고리즘

In [2]:
from mlxtend.preprocessing import TransactionEncoder

data = np.array([
    ['우유', '기저귀', '쥬스'],
    ['양상추', '기저귀', '맥주'],
    ['우유', '양상추', '기저귀', '맥주'],
    ['양상추', '맥주']
])

In [3]:
te = TransactionEncoder()
te_ary = te.fit(data).transform(data)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,기저귀,맥주,양상추,우유,쥬스
0,True,False,False,True,True
1,True,True,True,False,False
2,True,True,True,True,False
3,False,True,True,False,False


In [4]:
%%time
from mlxtend.frequent_patterns import apriori

apriori(df, min_support=0.5, use_colnames=True)

CPU times: user 15.6 ms, sys: 2.3 ms, total: 17.9 ms
Wall time: 22.3 ms


,support,itemsets
0,0.75,(기저귀)
1,0.75,(맥주)
2,0.75,(양상추)
3,0.50,(우유)
4,0.50,"(맥주, 기저귀)"
5,0.50,"(양상추, 기저귀)"
6,0.50,"(우유, 기저귀)"
7,0.75,"(양상추, 맥주)"
8,0.50,"(양상추, 맥주, 기저귀)"


## 2. FP-Growth 알고리즘

In [5]:
data = np.array([
    ['우유', '기저귀', '쥬스'],
    ['양상추', '기저귀', '맥주'],
    ['우유', '양상추', '기저귀', '맥주'],
    ['양상추', '맥주']
])

In [6]:
te = TransactionEncoder()
te_ary = te.fit(data).transform(data)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,기저귀,맥주,양상추,우유,쥬스
0,True,False,False,True,True
1,True,True,True,False,False
2,True,True,True,True,False
3,False,True,True,False,False


In [7]:
%%time
from mlxtend.frequent_patterns import fpgrowth

fpgrowth(df, min_support=0.5, use_colnames=True)

CPU times: user 4.07 ms, sys: 2 µs, total: 4.07 ms
Wall time: 3.83 ms


,support,itemsets
0,0.75,(기저귀)
1,0.50,(우유)
2,0.75,(양상추)
3,0.75,(맥주)
4,0.50,"(맥주, 기저귀)"
5,0.50,"(양상추, 기저귀)"
6,0.50,"(양상추, 맥주, 기저귀)"
7,0.50,"(우유, 기저귀)"
8,0.75,"(양상추, 맥주)"


## 3. TF-IDF 알고리즘

In [8]:
docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
] 

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
countvect = vect.fit_transform(docs)
countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))
countvect_df.index = ['문서1', '문서2', '문서3', '문서4']
countvect_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문서1,0,0,0,1,0,1,1,0,0
문서2,0,0,0,1,1,0,1,0,0
문서3,0,1,1,0,2,0,0,0,0
문서4,1,0,0,0,0,0,0,1,1


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidv = TfidfVectorizer(use_idf=True, smooth_idf=False, norm=None).fit(docs)
tfidv_df = pd.DataFrame(tfidv.transform(docs).toarray(), columns = sorted(tfidv.vocabulary_))
tfidv_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,1.693147,0.000000,2.386294,1.693147,0.000000,0.000000
1,0.000000,0.000000,0.000000,1.693147,1.693147,0.000000,1.693147,0.000000,0.000000
2,0.000000,2.386294,2.386294,0.000000,3.386294,0.000000,0.000000,0.000000,0.000000
3,2.386294,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.386294,2.386294


In [11]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(tfidv_df, tfidv_df)

array([[1.        , 0.57833696, 0.        , 0.        ],
       [0.57833696, 1.        , 0.40894599, 0.        ],
       [0.        , 0.40894599, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

## 4. Word2Vec 알고리즘

In [12]:
from gensim.models import Word2Vec

docs = [
  'you say goodbye and I say hello .'
]

sentences = [list(sentence.split(' ')) for sentence in docs]
sentences

[['you', 'say', 'goodbye', 'and', 'I', 'say', 'hello', '.']]

In [13]:
model = Word2Vec(size=3, window=1, min_count=1, sg=1)
model.build_vocab(sentences)
model.wv.most_similar("say")

[('I', 0.6964418888092041),
 ('.', 0.599804162979126),
 ('you', 0.3982182741165161),
 ('and', 0.24110147356987),
 ('hello', -0.6183485388755798),
 ('goodbye', -0.7039119005203247)]

## 5. KNN 알고리즘

In [22]:
import surprise
from surprise.model_selection import KFold
from surprise.model_selection import cross_validate
from surprise import Reader, Dataset, SVD, SVDpp, NMF, KNNBaseline
from surprise.model_selection import KFold
from surprise.model_selection import cross_validate

data = Dataset.load_builtin('ml-100k')
df = pd.DataFrame(data.raw_ratings, columns=["user", "item", "rate", "id"])
df = df.astype(np.float32)

del df["id"]
df.head(10)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


,user,item,rate
0,196.0,242.0,3.0
1,186.0,302.0,3.0
2,22.0,377.0,1.0
3,244.0,51.0,2.0
4,166.0,346.0,1.0
5,298.0,474.0,4.0
6,115.0,265.0,2.0
7,253.0,465.0,5.0
8,305.0,451.0,3.0
9,6.0,86.0,3.0


In [23]:
%%time 
reader = Reader(rating_scale=(1, 5))
knndata = Dataset.load_from_df(df[['user', 'item', 'rate']], reader)

sim_options = {'name': 'cosine'}
knn = surprise.KNNBasic(sim_options=sim_options, k=20)
score = cross_validate(knn, knndata, measures=['RMSE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0200  1.0226  1.0205  1.0266  1.0280  1.0235  0.0032  
Fit time          1.53    1.71    1.54    1.54    1.62    1.59    0.07    
Test time         5.72    5.61    5.56    5.54    5.52    5.59    0.07    
CPU times: user 37.1 s, sys: 209 ms, total: 37.3 s
Wall time: 37.3 s


In [24]:
%%time 
user = 196

score_dict = {}
for sim in knn.get_neighbors(user, k=20):
    df_ = df[df['user'] == sim]
    for item, rate in zip(df_['item'].values, df_['rate'].values):
        if item not in df[df['user'] == user]['item'].values:
            try:
                score_dict[item] += rate
            except:
                score_dict[item] = rate

CPU times: user 2.67 s, sys: 256 ms, total: 2.93 s
Wall time: 2.29 s


In [25]:
# 상위 10개의 영화만 추천 
dict(sorted(score_dict.items(), key = lambda x: -x[1])[0:10]).keys()

dict_keys([288.0, 258.0, 50.0, 98.0, 313.0, 127.0, 300.0, 100.0, 121.0, 272.0])

## 6. SGD 알고리즘

해당 코드는 https://yamalab.tistory.com/92 에 있는 Y.LAB의 블로그 글을 참고했습니다. (대부분의 코드가 같고 중간에 한 부분만 수정했습니다.)

In [26]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm_notebook as tqdm

import numpy as np

# Base code : https://yamalab.tistory.com/92
class MatrixFactorization():
    def __init__(self, R, k, learning_rate, reg_param, epochs, verbose=False):
        """
        :param R: rating matrix
        :param k: latent parameter
        :param learning_rate: alpha on weight update
        :param reg_param: beta on weight update
        :param epochs: training epochs
        :param verbose: print status
        """

        self._R = R
        self._num_users, self._num_items = R.shape
        self._k = k
        self._learning_rate = learning_rate
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose


    def fit(self):
        """
        training Matrix Factorization : Update matrix latent weight and bias

        참고: self._b에 대한 설명
        - global bias: input R에서 평가가 매겨진 rating의 평균값을 global bias로 사용
        - 정규화 기능. 최종 rating에 음수가 들어가는 것 대신 latent feature에 음수가 포함되도록 해줌.

        :return: training_process
        """

        # init latent features
        self._P = np.random.normal(size=(self._num_users, self._k))
        self._Q = np.random.normal(size=(self._num_items, self._k))

        # init biases
        self._b_P = np.zeros(self._num_users)
        self._b_Q = np.zeros(self._num_items)
        self._b = np.mean(self._R[np.where(self._R != 0)])

        # train while epochs
        self._training_process = []
        for epoch in range(self._epochs):
            # rating이 존재하는 index를 기준으로 training
            xi, yi = self._R.nonzero()
            for i, j in zip(xi, yi):
                self.gradient_descent(i, j, self._R[i, j])
            cost = self.cost()
            self._training_process.append((epoch, cost))

            # print status
            if self._verbose == True and ((epoch + 1) % 10 == 0):
                print("Iteration: %d ; cost = %.4f" % (epoch + 1, cost))


    def cost(self):
        """
        compute root mean square error
        :return: rmse cost
        """

        # xi, yi: R[xi, yi]는 nonzero인 value를 의미한다.
        # 참고: http://codepractice.tistory.com/90
        xi, yi = self._R.nonzero()
        # predicted = self.get_complete_matrix()
        cost = 0
        for x, y in zip(xi, yi):
            cost += pow(self._R[x, y] - self.get_prediction(x, y), 2)
        return np.sqrt(cost/len(xi))


    def gradient(self, error, i, j):
        """
        gradient of latent feature for GD

        :param error: rating - prediction error
        :param i: user index
        :param j: item index
        :return: gradient of latent feature tuple
        """

        dp = (error * self._Q[j, :]) - (self._reg_param * self._P[i, :])
        dq = (error * self._P[i, :]) - (self._reg_param * self._Q[j, :])
        return dp, dq


    def gradient_descent(self, i, j, rating):
        """
        graident descent function

        :param i: user index of matrix
        :param j: item index of matrix
        :param rating: rating of (i,j)
        """

        # get error
        prediction = self.get_prediction(i, j)
        error = rating - prediction

        # update biases
        self._b_P[i] += self._learning_rate * (error - self._reg_param * self._b_P[i])
        self._b_Q[j] += self._learning_rate * (error - self._reg_param * self._b_Q[j])

        # update latent feature
        dp, dq = self.gradient(error, i, j)
        self._P[i, :] += self._learning_rate * dp
        self._Q[j, :] += self._learning_rate * dq


    def get_prediction(self, i, j):
        """
        get predicted rating: user_i, item_j
        :return: prediction of r_ij
        """
        return self._b + self._b_P[i] + self._b_Q[j] + self._P[i, :].dot(self._Q[j, :].T)


    def get_complete_matrix(self):
        """
        computer complete matrix PXQ + P.bias + Q.bias + global bias

        - PXQ 행렬에 b_P[:, np.newaxis]를 더하는 것은 각 열마다 bias를 더해주는 것
        - b_Q[np.newaxis:, ]를 더하는 것은 각 행마다 bias를 더해주는 것
        - b를 더하는 것은 각 element마다 bias를 더해주는 것

        - newaxis: 차원을 추가해줌. 1차원인 Latent들로 2차원의 R에 행/열 단위 연산을 해주기위해 차원을 추가하는 것.

        :return: complete matrix R^
        """
        return self._b + self._b_P[:, np.newaxis] + self._b_Q[np.newaxis:, ] + self._P.dot(self._Q.T)



# run example
if __name__ == "__main__":
    # rating matrix - User X Item : (7 X 5)
    R = np.array([
        [1, 0, 0, 1, 3],
        [2, 0, 3, 1, 1],
        [1, 2, 0, 5, 0],
        [1, 0, 0, 4, 4],
        [2, 1, 5, 4, 0],
        [5, 1, 5, 4, 0],
        [0, 0, 0, 1, 0],
    ])

    # P, Q is (7 X k), (k X 5) matrix
    

In [16]:
%%time
factorizer = MatrixFactorization(R, k=3, learning_rate=0.01, reg_param=0.01, epochs=100, verbose=True)
factorizer.fit()

Iteration: 10 ; cost = 1.1216
Iteration: 20 ; cost = 0.8614
Iteration: 30 ; cost = 0.7214
Iteration: 40 ; cost = 0.6159
Iteration: 50 ; cost = 0.5187
Iteration: 60 ; cost = 0.4270
Iteration: 70 ; cost = 0.3453
Iteration: 80 ; cost = 0.2767
Iteration: 90 ; cost = 0.2211
Iteration: 100 ; cost = 0.1766
CPU times: user 149 ms, sys: 41.1 ms, total: 190 ms
Wall time: 151 ms


In [17]:
factorizer.get_complete_matrix()

array([[1.14066167, 1.38455162, 2.75412761, 1.09773331, 2.85248466],
       [1.79281788, 2.11382079, 2.63050136, 1.13155136, 1.38821559],
       [0.98531183, 1.94135412, 5.29466311, 4.8776737 , 3.2275958 ],
       [1.30701729, 0.94028454, 4.74660051, 4.13356665, 3.71827928],
       [1.87739112, 1.01462608, 5.01812121, 3.93500871, 5.71843861],
       [4.84237327, 1.1475143 , 4.94600711, 4.02501394, 4.8115378 ],
       [3.83590178, 2.50209319, 2.8303019 , 0.91282592, 2.73312068]])

## 7. ALS 알고리즘

In [18]:
from implicit.evaluation import *
from implicit.als import AlternatingLeastSquares as ALS

In [27]:
# Implicit data
# 예시를 위해서 rate의 값을 1로 변경해주었습니다. 
df['rate'] = 1

In [28]:
user2idx = {}
for i, l in enumerate(df['user'].unique()):
    user2idx[l] = i
    
movie2idx = {}
for i, l in enumerate(df['item'].unique()):
    movie2idx[l] = i

In [29]:
idx2user = {i: user for user, i in user2idx.items()}
idx2movie = {i: item for item, i in movie2idx.items()}

In [30]:
useridx = df['useridx'] = df['user'].apply(lambda x: user2idx[x]).values
movieidx = df['movieidx'] = df['item'].apply(lambda x: movie2idx[x]).values
rating = df['rate'].values

In [31]:
import scipy

purchase_sparse = scipy.sparse.csr_matrix((rating, (useridx, movieidx)), shape=(len(set(useridx)), len(set(movieidx))))

In [32]:
als_model = ALS(factors=20, regularization=0.08, iterations = 20)
als_model.fit(purchase_sparse.T)

In [33]:
als_model.recommend(0, purchase_sparse, N=150)[0:10]

[(33, 0.38986203),
 (166, 0.3859308),
 (366, 0.34752324),
 (133, 0.33900058),
 (96, 0.32630354),
 (226, 0.30503827),
 (89, 0.3017478),
 (404, 0.2944044),
 (148, 0.29051706),
 (600, 0.28714183)]